In [89]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

pd.options.display.max_rows = 999
pd.set_option('max_colwidth', 100)

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


#Data Preprocessing and Feature Engineering
from textblob import TextBlob
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

#Model Selection and Validation
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report,accuracy_score


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [90]:
data = pd.read_csv('/content/train_2kmZucJ.csv')

test = pd.read_csv('/content/test_oJQbWVk.csv')
test.head()
test['label'] = "1"
test_data1 = test.iloc[:,1:3]
test_data1.head()

,tweet,label
0,I hate the new #iphone upgrade. Won't let me download apps. #ugh #apple sucks,1
1,currently shitting my fucking pants. #apple #iMac #cashmoney #raddest #swagswagswag http://insta...,1
2,"I'd like to puts some CD-ROMS on my iPad, is that possible?' — Yes, but wouldn't that block the ...",1
3,"My ipod is officially dead. I lost all my pictures and videos from the 1D and 5sos concert,and f...",1
4,Been fighting iTunes all night! I only want the music I $&@*# paid for,1


In [0]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)


reviews = []
sentences = list(data['tweet'])
for sen in sentences:
    reviews.append(preprocess_text(sen))


clean_data = pd.DataFrame(reviews,columns=['tweet'])
clean_data['label'] = data.label


In [92]:
!pip install bert-for-tf2
!pip install sentencepiece
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [93]:
y = clean_data['label']
print(reviews[0])
y[0]

 fingerprint Pregnancy Test https goo gl MfQV android apps beautiful cute health igers iphoneonly iphonesia iphone


0

In [0]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [0]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [0]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

In [97]:
tokenized_reviews[1]

[2633,
 9099,
 28689,
 3372,
 13773,
 2553,
 4283,
 2000,
 2026,
 4470,
 8038,
 2100,
 8412,
 26726,
 11610,
 8412,
 10288,
 4842,
 7951,
 8299,
 16021,
 23091,
 4012,
 1061,
 18150,
 29175,
 1046,
 2213]

In [98]:
reviews_with_len = [[review, y[i], len(review)]
                 for i, review in enumerate(tokenized_reviews)]

sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]


sorted_reviews_labels[1:6]

[([2633,
   9099,
   28689,
   3372,
   13773,
   2553,
   4283,
   2000,
   2026,
   4470,
   8038,
   2100,
   8412,
   26726,
   11610,
   8412,
   10288,
   4842,
   7951,
   8299,
   16021,
   23091,
   4012,
   1061,
   18150,
   29175,
   1046,
   2213],
  0),
 ([2057,
   2293,
   2023,
   2052,
   2017,
   2175,
   2831,
   2191,
   4168,
   24610,
   2229,
   4895,
   24759,
   15916,
   9483,
   18059,
   26381,
   15536,
   8873,
   7532,
   8299,
   1042,
   2497,
   2033,
   21849,
   15042,
   2226],
  0),
 ([1045,
   17502,
   2113,
   1049,
   2577,
   2001,
   2081,
   2008,
   2126,
   18059,
   10140,
   4913,
   3372,
   2854,
   2188,
   8299,
   16021,
   15900,
   2099,
   2572,
   5622,
   1057,
   22578],
  0),
 ([2054,
   6429,
   2326,
   6207,
   2180,
   2130,
   2831,
   2000,
   2033,
   2055,
   3160,
   2031,
   4983,
   3477,
   2068,
   2005,
   2037,
   5236,
   2490],
  1),
 ([18059, 4007, 10651, 21746, 2039, 2026, 3042, 2502, 2051, 5236, 18059, 201

In [0]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

In [0]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [101]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 53), dtype=int32, numpy=
 array([[ 4344, 16550, 10032, ...,     0,     0,     0],
        [ 2633,  9099, 28689, ...,     0,     0,     0],
        [ 2057,  2293,  2023, ...,     0,     0,     0],
        ...,
        [ 2442,  3422,  7858, ...,     0,     0,     0],
        [ 4067,  2017, 19102, ...,     0,     0,     0],
        [10381,  8854,  9790, ..., 13958,  8197,  1039]], dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
        1, 1, 0, 0, 1, 1, 0, 0, 0, 0], dtype=int32)>)

In [102]:
import math
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
print(TOTAL_BATCHES)

248


In [103]:
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
print(TEST_BATCHES)
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)


24


In [0]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [0]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 4


In [106]:
text_model = TEXT_MODEL(vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)
text_model.save_weights("/content/model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
if OUTPUT_CLASSES == 2:
    text_model.compile(loss="binary_crossentropy",
                       optimizer="adam",
                       metrics=["accuracy"])
else:
    text_model.compile(loss="sparse_categorical_crossentropy",
                       optimizer="adam",
                       metrics=["sparse_categorical_accuracy"])

In [108]:
text_model.fit(train_data, epochs=NB_EPOCHS)

Epoch 1/4
224/224 [==============================] - 26s 117ms/step - loss: 0.3118 - accuracy: 0.8588
Epoch 2/4
224/224 [==============================] - 26s 118ms/step - loss: 0.1555 - accuracy: 0.9393
Epoch 3/4
224/224 [==============================] - 26s 118ms/step - loss: 0.0731 - accuracy: 0.9769
Epoch 4/4
224/224 [==============================] - 26s 117ms/step - loss: 0.0325 - accuracy: 0.9920


In [109]:
text_model.summary()

Model: "text_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      multiple                  6104400   
_________________________________________________________________
conv1d_12 (Conv1D)           multiple                  40100     
_________________________________________________________________
conv1d_13 (Conv1D)           multiple                  60100     
_________________________________________________________________
conv1d_14 (Conv1D)           multiple                  80100     
_________________________________________________________________
global_max_pooling1d_4 (Glob multiple                  0         
_________________________________________________________________
dense_8 (Dense)              multiple                  77056     
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0

In [110]:
results = text_model.predict(test_data)
test_data
# print(results)

<TakeDataset shapes: ((None, None), (None,)), types: (tf.int32, tf.int32)>

In [0]:
def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)


reviews = []
sentences = list(test_data1['tweet'])
for sen in sentences:
    reviews.append(preprocess_text(sen))


clean_data = pd.DataFrame(reviews,columns=['tweet'])
clean_data['label'] = test_data1.label


In [112]:
clean_data.head()

,tweet,label
0,I hate the new iphone upgrade Won let me download apps ugh apple sucks,1
1,currently shitting my fucking pants apple iMac cashmoney raddest swagswagswag http instagr am UU...,1
2,I like to puts some CD ROMS on my iPad is that possible Yes but wouldn that block the screen,1
3,My ipod is officially dead lost all my pictures and videos from the and sos concert and from Vet...,1
4,Been fighting iTunes all night only want the music paid for,1


In [0]:
def tokenize_reviews(text_reviews):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text_reviews))

In [0]:
tokenized_reviews = [tokenize_reviews(review) for review in reviews]

In [115]:
tokenized_reviews[1]

[2747,
 4485,
 3436,
 2026,
 8239,
 6471,
 6207,
 10047,
 6305,
 5356,
 8202,
 3240,
 10958,
 14141,
 4355,
 25430,
 26454,
 4213,
 5620,
 4213,
 2290,
 8299,
 16021,
 15900,
 2099,
 2572,
 1057,
 27020,
 12170,
 2497,
 6844]

In [116]:
reviews_with_len = [[review, y[i], len(review)]
                 for i, review in enumerate(tokenized_reviews)]

sorted_reviews_labels = [(review_lab[0], review_lab[1]) for review_lab in reviews_with_len]


sorted_reviews_labels[1:6]

[([2747,
   4485,
   3436,
   2026,
   8239,
   6471,
   6207,
   10047,
   6305,
   5356,
   8202,
   3240,
   10958,
   14141,
   4355,
   25430,
   26454,
   4213,
   5620,
   4213,
   2290,
   8299,
   16021,
   15900,
   2099,
   2572,
   1057,
   27020,
   12170,
   2497,
   6844],
  0),
 ([1045,
   2066,
   2000,
   8509,
   2070,
   3729,
   17083,
   2015,
   2006,
   2026,
   25249,
   2003,
   2008,
   2825,
   2748,
   2021,
   2876,
   2008,
   3796,
   1996,
   3898],
  0),
 ([2026,
   26322,
   2003,
   3985,
   2757,
   2439,
   2035,
   2026,
   4620,
   1998,
   6876,
   2013,
   1996,
   1998,
   2061,
   2015,
   4164,
   1998,
   2013,
   29525,
   3409,
   22650,
   15509,
   20040],
  0),
 ([2042, 3554, 11943, 2035, 2305, 2069, 2215, 1996, 2189, 3825, 2005], 1),
 ([16360,
   14122,
   2131,
   3676,
   8126,
   2078,
   19699,
   6340,
   2007,
   16360,
   28696,
   9397,
   2000,
   14970,
   2057,
   2097,
   2031,
   2256,
   6207,
   15477,
   16770,
   7479

In [0]:
processed_dataset = tf.data.Dataset.from_generator(lambda: sorted_reviews_labels, output_types=(tf.int32, tf.int32))

In [118]:
processed_dataset

<FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>

In [0]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [120]:
next(iter(batched_dataset))

(<tf.Tensor: shape=(32, 62), dtype=int32, numpy=
 array([[1045, 5223, 1996, ...,    0,    0,    0],
        [2747, 4485, 3436, ...,    0,    0,    0],
        [1045, 2066, 2000, ...,    0,    0,    0],
        ...,
        [6823, 3238, 2074, ...,    0,    0,    0],
        [1045, 2123, 3342, ...,    0,    0,    0],
        [2097, 3889, 5558, ...,    0,    0,    0]], dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0,
        1, 1, 0, 0, 1, 1, 0, 0, 0, 0], dtype=int32)>)

In [0]:
TOTAL_BATCHES = math.ceil(len(sorted_reviews_labels) / BATCH_SIZE)

In [0]:
test_data1 = batched_dataset.take(TOTAL_BATCHES)

In [123]:
results = text_model.predict(test_data1)
results

array([[9.9634618e-01],
       [9.7315490e-01],
       [9.9901271e-01],
       ...,
       [9.8292041e-01],
       [9.7277427e-01],
       [4.5615295e-08]], dtype=float32)

In [125]:
type(results)

numpy.ndarray

In [0]:
A = results.astype(int)

In [140]:
A

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [0]:
np.savetxt("foo.csv", A, delimiter=",")